Aqui montaremos a rede para avaliar os dados para a tarefa de classificação multiclasse com a extração de características hog (Histogram of Oriented Gradients)

Importar as bibliotecas e os dados do conjunto MNIST

In [1]:
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler

# 1. Carregar o conjunto de dados MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Criando a função que implementa o descritor HOG

In [2]:
def extract_hog_features(images):
    features = []
    for image in images:
        fd = hog(image, orientations=8, pixels_per_cell=(4, 4), cells_per_block=(1, 1), visualize=False)
        features.append(fd)
    return np.array(features)

Normalizando os dados. As imagens já vem em escala de cinza.

In [3]:
x_train = x_train / 255.0
x_test = x_test / 255.0

Extraír as características HOG do conjunto

In [4]:
x_train_hog = extract_hog_features(x_train)
x_test_hog = extract_hog_features(x_test)

Normalizando as características HOG extraídas

In [5]:
scaler = StandardScaler()
x_train_hog = scaler.fit_transform(x_train_hog)
x_test_hog = scaler.transform(x_test_hog)

Redimensionando os dados normalizados, extraídos com HOG, para o formato esperado pela CNN

In [6]:
x_train_hog = x_train_hog.reshape(x_train_hog.shape[0], 1, 1, x_train_hog.shape[1])
x_test_hog = x_test_hog.reshape(x_test_hog.shape[0], 1, 1, x_test_hog.shape[1])

Converter os rótulos para one-hot encoding

`to_categorical(rótulos, número de classes)`

In [7]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

Construir o modelo

In [8]:
model = Sequential([
    Flatten(input_shape=(x_train_hog.shape[1], x_train_hog.shape[2], x_train_hog.shape[3])),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

c:\Users\mtoma\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Compilar o modelo

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

- Criando e salvando os hiperparâmetros da arquitetura e da inicialização em formato json em arquivos-hog/hiperparametros.json

- Salvando os pesos iniciais

In [10]:
# Definindo os hiperparâmetros
hiperparametros = {
    "arquitetura": {
        "camadas": [64, 64, 10],
        "ativacao": "relu",
        "ultimo_ativacao": "softmax"
    },
    "inicializacao": {
        "pesos": "he_normal",
        "bias": "zeros"
    }
}

# Serializando os hiperparâmetros em uma string JSON
hiperparametros_json = json.dumps(hiperparametros, indent=4)

# Escrevendo a string JSON em um arquivo
with open("arquivos-hog/hiperparametros.json", "w") as arquivo:
    arquivo.write(hiperparametros_json)


# PESOS INICIAIS
model.save_weights('arquivos-hog/pesos_iniciais.weights.h5')

Treinar o modelo

In [11]:
historico = model.fit(x_train_hog, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8171 - loss: 0.5838 - val_accuracy: 0.9692 - val_loss: 0.1052
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9513 - loss: 0.1567 - val_accuracy: 0.9722 - val_loss: 0.0970
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9619 - loss: 0.1246 - val_accuracy: 0.9732 - val_loss: 0.1047
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9675 - loss: 0.1063 - val_accuracy: 0.9752 - val_loss: 0.1168
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9689 - loss: 0.0991 - val_accuracy: 0.9762 - val_loss: 0.1108
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9725 - loss: 0.0877 - val_accuracy: 0.9753 - val_loss: 0.1156
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9733 - loss: 0.0823 - val_accuracy: 0.9755 - val_loss: 0.1084
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9758 - loss: 0.0767 - 

- Salvando os pesos finais da rede 
- Salvando o histórico de perda para cada iteração
- Salvando as saídas produzidas pela rede para cada um dos dados de teste

In [12]:
# PESOS FINAIS
model.save_weights('arquivos-hog/pesos_finais.weights.h5')

# ERRO DE CADA ITERAÇÃO
perdas = historico.history['loss']

# Salvando o histórico de perda em um arquivo JSON
with open('arquivos-hog/historico_perda.json', 'w') as f:
    json.dump(perdas, f)


# SAÍDAS PRODUZIDAS
# Fazendo inferência com o modelo treinado para obter as saídas
saidas = model.predict(x_train_hog)

# Convertendo as saídas para uma lista para serialização
saidas_lista = saidas.tolist()

# Salvando as saídas em um arquivo JSON
with open('arquivos-hog/saidas_teste.json', 'w') as f:
    json.dump(saidas_lista, f)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step


Testar o modelo

In [13]:
test_loss, test_acc = model.evaluate(x_test_hog, y_test)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9723 - loss: 0.0959
Test accuracy: 0.9781000018119812, Test loss: 0.07785646617412567
